In [ ]:
import json
import pandas as pd

# BUILD ALL CSV FILES

In [ ]:
# Build csv files for all xps

sender_type = ['sender_rdma', 'sender_rdma_write_multip_write', 'sender_rdma_write_multip_read', 'sender_rdma_write_nomultip_read']
payload_sizes = ['10-100','100-1000','1000-10000']
simult_req = ['1','10','100']
payload_bounds = ['1500']
circle_sizes = ['200']

global_data = []

for size in payload_sizes:
   for n in simult_req:
    for payload_bound in payload_bounds:
            for circle_size in circle_sizes:
        # one xp, let's get th latencies and avg req rates
        #   CASE   OPS/SEC ...
        # sender | res1 res2 res3        
                data = []
                #For custom senders
                for sender in sender_type:
                    result_json = f"results/json_{sender}_{size}_{n}_{payload_bound}_{circle_size}"
                    # print(result_json)            
                    with open(result_json, 'r') as file:
                        result =  json.load(file)                
                        
                        result_dict = {}
                        result_dict["sender_type"] = sender[7:]
                        result_dict["payload_size"] = size
                        result_dict["simult_req"] = n
                        result_dict["payload_bound"] = payload_bound
                        result_dict["circle_size"] = circle_size
                        result_dict["Ops/sec"] = result['ALL STATS']['Totals']['Ops/sec']
                        result_dict["Average"] = result['ALL STATS']['Totals']['Average Latency']
                        result_dict["p50.00"] = result['ALL STATS']['Totals']['Percentile Latencies']['p50.00']
                        result_dict["p99.00"] = result['ALL STATS']['Totals']['Percentile Latencies']['p99.00']
                        result_dict["p99.90"] = result['ALL STATS']['Totals']['Percentile Latencies']['p99.90']
                        
                        data.append(result_dict)
                        global_data.append(result_dict)
                    # print(result)  

                #For tcp_proxy    
                result_json = f"results/json_tcpproxy_{size}_{n}"
                # print(result_json)            
                with open(result_json, 'r') as file:
                    result =  json.load(file)                
                    
                    result_dict = {}
                    result_dict["sender_type"] = "tcpproxy"
                    result_dict["payload_size"] = size
                    result_dict["simult_req"] = n
                    result_dict["payload_bound"] = payload_bound
                    result_dict["circle_size"] = circle_size
                    result_dict["Ops/sec"] = result['ALL STATS']['Totals']['Ops/sec']
                    result_dict["Average"] = result['ALL STATS']['Totals']['Average Latency']
                    result_dict["p50.00"] = result['ALL STATS']['Totals']['Percentile Latencies']['p50.00']
                    result_dict["p99.00"] = result['ALL STATS']['Totals']['Percentile Latencies']['p99.00']
                    result_dict["p99.90"] = result['ALL STATS']['Totals']['Percentile Latencies']['p99.90']
                    
                    data.append(result_dict)
                    global_data.append(result_dict)
                # print(result)  

                # print("\n\n\n")
                df = pd.DataFrame(data)
                df.to_csv(f'data/res_{size}_{n}_{payload_bound}_{circle_size}.csv')

global_df = pd.DataFrame(global_data)
global_df.to_csv(f'data/res_global.csv')

# Build individual metric comparison plots
1 dimension x axis

In [ ]:
# Build individual metric comparison plots

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

payload_sizes = ['10-100','100-1000','1000-10000']
simult_req = ['1','10','100']
payload_bounds = ['1500']
circle_sizes = ['200']
metrics = ['p99.90', 'Average']
for size in payload_sizes:
   for n in simult_req:
    for payload_bound in payload_bounds:
            for circle_size in circle_sizes:
                for metric in metrics:
                    plt.clf() # Reset plot

                    # Read the CSV file
                    df = pd.read_csv(f"data/res_{size}_{n}_{payload_bound}_{circle_size}.csv")

                    # Extract the required data
                    sender_types = df['sender_type']
                    values = df[metric]

                    # Set up seaborn style
                    sns.set(style="whitegrid")

                    # Create a bar plot
                    plt.figure(figsize=(10, 6))  # Adjust the figure size as needed
                    sns.barplot(x=sender_types, y=values, palette="viridis")
                    plt.xlabel('Prototype', fontsize=12)
                    plt.ylabel(f'{metric} Latency (ms)', fontsize=12)
                    plt.title(f"""Comparison of {metric} Latency by Prototype\n
                                for payload_size={size}, simult_req={n}, payload_bound={payload_bound}, circle_size={circle_size}"""
                            , fontsize=14)
                    plt.xticks(rotation=45, ha='right', fontsize=10)  # Rotate and align x-axis labels

                    # Add data labels
                    for index, value in enumerate(values):
                        plt.text(index, value, str(round(value, 3)), ha='center', va='bottom', fontsize=9)

                    # Remove top and right spines
                    sns.despine()

                    # Display the plot
                    plt.tight_layout()  # Adjust spacing
                    
                    plt.savefig(f'plots/type1_{size}_{n}_{payload_bound}_{circle_size}_{metric}.svg', bbox_inches='tight', format='svg')

# Build (sender_type, param) comparison plots
2 dimensions x axis

In [ ]:
# Build (sender_type, payload_size) comparison plots

import pandas as pd
import matplotlib.pyplot as plt

# Read the CSV file into a pandas DataFrame
df = pd.read_csv('data/res_global.csv')

simult_req = [1, 10, 100]
payload_bounds = [1500]
circle_sizes = [200]
metrics = ['p99.90', 'p99.00', 'Average']

# Set plot style
plt.style.use('seaborn-whitegrid')

# Compare average throughput
for n in simult_req:
    for payload_bound in payload_bounds:
        for circle_size in circle_sizes:
            for metric in metrics:
                plt.clf() # Reset plot
                df_fixed = df[(df['simult_req'] == n) & (df['payload_bound'] == payload_bound) & (df['circle_size'] == circle_size)]

                df_comp_fixed_temp = df_fixed.pivot("sender_type", "payload_size", metric)
                ax = df_comp_fixed_temp.plot(kind='bar', rot=0)

                # Set plot title and labels
                plt.title(f"{metric} Latency comparison \n for simult_req={n}, payload_bound={payload_bound}, circle_size={circle_size}")
                plt.xlabel('Protoype')
                plt.ylabel(f"{metric} Latency (ms)")

                # Orient sender_type labels at 45 degrees
                ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
                ax.tick_params(axis='x', pad=10)

                # Adjust plot layout
                plt.legend(title='Payload Size', bbox_to_anchor=(1, 1))
                plt.tight_layout()

                # Show or save the plot
                plt.savefig(f'plots/sender-comp-payload_size-{n}_{payload_bound}_{circle_size}_{metric}.svg', bbox_inches='tight', format='svg')

In [ ]:
# Build (sender_type, simult_req) comparison plots

import pandas as pd
import matplotlib.pyplot as plt

# Read the CSV file into a pandas DataFrame
df = pd.read_csv('data/res_global.csv')

payload_sizes = ['10-100','100-1000','1000-10000']
payload_bounds = [1500]
circle_sizes = [200]
metrics = ['p99.90', 'p99.00', 'Average']

# Set plot style
plt.style.use('seaborn-whitegrid')

# Compare average throughput
for size in payload_sizes:
    for payload_bound in payload_bounds:
        for circle_size in circle_sizes:
            for metric in metrics:
                plt.clf() # Reset plot
                df_fixed = df[(df['payload_size'] == size) & (df['payload_bound'] == payload_bound) & (df['circle_size'] == circle_size)]

                df_comp_fixed_temp = df_fixed.pivot("sender_type", "simult_req", metric)
                ax = df_comp_fixed_temp.plot(kind='bar', rot=0)

                # Set plot title and labels
                plt.title(f"{metric} Latency comparison \n for payload_size={size}, payload_bound={payload_bound}, circle_size={circle_size}")
                plt.xlabel('Protoype')
                plt.ylabel(f"{metric} Latency (ms)")

                # Orient sender_type labels at 45 degrees
                ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
                ax.tick_params(axis='x', pad=10)

                # Adjust plot layout
                plt.legend(title='Simultaneous Requests', bbox_to_anchor=(1, 1))
                plt.tight_layout()

                # Show or save the plot
                plt.savefig(f'plots/sender-comp-simult_req-{size}_{payload_bound}_{circle_size}_{metric}.svg', bbox_inches='tight', format='svg')

# Build "over time" plots

In [ ]:
# Used for "over time" plots

import matplotlib.pyplot as plt

def plot_metric_over_time(time_series, metric_name, duration, y_axis_name, plot_filename):
    metric_values_over_time = []
    for t in range(duration):
        metric_values_over_time.append(time_series[str(t)][metric_name])

    plt.clf()
    plt.bar(range(duration), metric_values_over_time)
    plt.xlabel('Time (s)', fontsize=12)
    plt.ylabel(y_axis_name, fontsize=12)
    plt.xticks(fontsize=10, rotation=45, ha='right')
    plt.yticks(fontsize=10)
    plt.title(f'{y_axis_name} over Time', fontsize=14)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.savefig(f"plots/{plot_filename}.svg")

In [ ]:

# Used for "over time" plots
# Generate plots for all json
sender_type = ['sender', 'sender_rdma', 'sender_rdma_write_multip_write', 'tcpproxy']
payload_sizes = ['10-100','100-1000','1000-10000']
simult_req = ['1','10','100']
for size in payload_sizes:
   for n in simult_req:     
        data = []
        for sender in sender_type:
            result_json = f"results/json_{sender}_{size}_{n}"
            print(result_json)
            with open(result_json, 'r') as file:
                result =  json.load(file)        

            duration = result['configuration']['test_time']
            time_series = result['ALL STATS']['Totals']['Time-Serie']

            plot_metric_over_time(time_series=time_series, metric_name='p99.90', duration=duration, y_axis_name='p99.90 Latency (ms)', plot_filename=f'{sender}_{size}_{n}_p99.90')
            plot_metric_over_time(time_series=time_series, metric_name='p99.00', duration=duration, y_axis_name='p99.00 Latency (ms)', plot_filename=f'{sender}_{size}_{n}_p99.00')
            plot_metric_over_time(time_series=time_series, metric_name='p50.00', duration=duration, y_axis_name='p50.00 Latency (ms)', plot_filename=f'{sender}_{size}_{n}_p50.00')
            plot_metric_over_time(time_series=time_series, metric_name='Average Latency', duration=duration, y_axis_name='Average Latency (ms)', plot_filename=f'{sender}_{size}_{n}_avg_latency')
            plot_metric_over_time(time_series=time_series, metric_name='Count', duration=duration, y_axis_name='Count', plot_filename=f'{sender}_{size}_{n}_count')
            plot_metric_over_time(time_series=time_series, metric_name='Max Latency', duration=duration, y_axis_name='Maximum Latency (ms)', plot_filename=f'{sender}_{size}_{n}_max_latency')

In [ ]:

# Generate plots for individual json
result_json = f"results/json_rdma_sender_10-100_1"
print(result_json)          

duration = result['configuration']['test_time']
plot_metric_over_time(time_series=time_series, metric_name='p99.90', duration=duration, y_axis_name='p99.90 Latency (ms)', plot_filename=f'{sender}_{size}_{n}_p99.90')
plot_metric_over_time(time_series=time_series, metric_name='p99.00', duration=duration, y_axis_name='p99.00 Latency (ms)', plot_filename=f'{sender}_{size}_{n}_p99.00')
plot_metric_over_time(time_series=time_series, metric_name='p50.00', duration=duration, y_axis_name='p50.00 Latency (ms)', plot_filename=f'{sender}_{size}_{n}_p50.00')
plot_metric_over_time(time_series=time_series, metric_name='Average Latency', duration=duration, y_axis_name='Average Latency (ms)', plot_filename=f'{sender}_{size}_{n}_avg_latency')
plot_metric_over_time(time_series=time_series, metric_name='Count', duration=duration, y_axis_name='Count', plot_filename=f'{sender}_{size}_{n}_count')
plot_metric_over_time(time_series=time_series, metric_name='Max Latency', duration=duration, y_axis_name='Maximum Latency (ms)', plot_filename=f'{sender}_{size}_{n}_max_latency')